### Generate M random potentials on [0, L] as sum of n gaussian kernels on G discretization points


there are 3 uniformly distributed random parameters a, b, c
the potential is calculated by:

$$
V(x) = -\sum_{n=1}^{3} a_n \exp \left(-\frac{(x - b_n)^2}{2c_n^2} \right)
$$

### Then solve the schroedinger equation in atomic units for 4 spinless non-interacting fermions.

The shooting numerov algorithm calculates a forward pass and a backward pass solution.

The final solution is then found by aligning the two passes and finding a good split point to concatenate them.

All integrals are approximated by the trapezoidal method.
The numerical differentiation for the laplacian uses the standard approximation.

### Usage:

Setting the dataset variable to different values will allow generating all the variations of the datasets:
standard, recreate, recreate/fine_0.125, ..., recreate/fine_10


In [ ]:
# setup notebook if it is run on Google Colab, cwd = notebook file location
try:
    # change notebook_path if this notebook is in a different subfolder of Google Drive
    notebook_path = "Projects/QuantumFlow/notebooks"

    import os
    from google.colab import drive
    drive.mount('/content/gdrive')
    os.chdir("/content/gdrive/My Drive/" + notebook_path)

    %tensorflow_version 2.x
    !pip install -q ruamel.yaml
except:
    pass

# imports
import tensorflow as tf

# setup paths and variables for shared code (../quantumflow) and data (../data)
import sys
sys.path.append('../')
data_dir = "../data"

# import shared code, must run 0_create_shared_project_files.ipynb first!
from quantumflow.generate_datasets import generate_potentials, generate_datasets

# Recreate Dataset

In [ ]:
experiment = "datasets"
base_dir = os.path.join(data_dir, experiment)
if not os.path.exists(base_dir): os.makedirs(base_dir)
file_hyperparams = os.path.join(base_dir, "hyperparams.config")

In [ ]:
%%writefile $file_hyperparams
globals: []

test: &DEFAULT
    filename: dataset_test
    format: hdf5 # pickle, hdf5
    seed: 0
    n_orbitals: 4 # N
    dataset_size: 1000 # M
    batch_size: 10000

    discretisation_points: 500 # G
    interval_length: 1.0 # [Bohr]
    n_gauss: 3 # number of gaussians 
    n_method: sum # sum or mean the gaussians
    a_minmax: [1.0, 10.0] # [hartree] 
    b_minmax: [0.4, 0.6] # nomalized x
    c_minmax: [0.03, 0.1] # nomalized x

    dtype: float64
    numerov_init_slope: 5.0

test_small:
    <<: *DEFAULT
    filename: dataset_test_small
    seed: 0
    dataset_size: 100

validate:
    <<: *DEFAULT
    filename: dataset_validate
    seed: -1
    dataset_size: 1000

fine:
    <<: *DEFAULT
    filename: dataset_fine
    discretisation_points: 4991

float:
    <<: *DEFAULT
    filename: dataset_float
    dtype: float32

medium: 
    <<: *DEFAULT
    filename: dataset_medium
    seed: 1
    dataset_size: 500

large:
    <<: *DEFAULT
    filename: dataset_large
    seed: 1
    dataset_size: 1000

larger:
    <<: *DEFAULT
    filename: dataset_larger
    seed: 1
    dataset_size: 10000

huge:
    <<: *DEFAULT
    filename: dataset_huge
    seed: 1
    dataset_size: 50000

huger:
    <<: *DEFAULT
    filename: dataset_huger
    seed: 1
    dataset_size: 100000

In [ ]:
generate_datasets(data_dir=data_dir, experiment='datasets', generate_names=['test', 'validate'])